# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 

Built by Dmitry Zaganich in July 2017
***
This is a system for detecting lane lines.
My lane detection pipeline was build as follows: 

1) Convert original image (video frame) to grayscale

2) Build masks for yellow and white pixels

3) Isolate yellow and white pixels  

4) Apply gaussian blur to supress the noise 

5) Apply a Canny edge detection  

6) Isolate "region of interest" in front of the vehicle  

7) Apply Hough transform and detect lines 

8) Use np.polyfit to identify the lanes 

9) Overlay lanes as two red lines over the original image (frame) 
***

## My implementation of the pipeline

Project dependencies

In [1]:
# project dependencies
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

Functions which will be used in the pipeline. 

I modified "draw_lines" so it classifies every line as part of right or left lane. After that, it uses np.polyfit with a degree of 1 to approximately find every lane. It also detects approximate height of every lane.

In [2]:
# utils
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=6):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    det_slope = 0.4
    
    right_lane_x, right_lane_y = [],[]
    left_lane_x, left_lane_y = [],[]
    
    imshape = img.shape
    
    right_top_y = imshape[0]
    left_top_y = imshape[0]
    
    for line in lines:
        #1
        for x1,y1,x2,y2 in line:
            slope = get_slope(x1,y1,x2,y2)
            if slope > det_slope:
                right_lane_x.append(x1)
                right_lane_y.append(y1)
                right_lane_x.append(x2)
                right_lane_y.append(y2)
                
                right_top_y = min(y1, y2, right_top_y)
            elif slope < -det_slope:
                left_lane_x.append(x1)
                left_lane_y.append(y1)
                left_lane_x.append(x2)
                left_lane_y.append(y2)
                
                left_top_y = min(y1, y2, left_top_y)
                
    global_top_y = min(right_top_y, left_top_y)            

    # draw simple interpolated lines
    right_lane_line = np.polyfit(right_lane_x, right_lane_y, 1)
    left_lane_line = np.polyfit(left_lane_x, left_lane_y, 1)
    
    right_lane_solve_top = np.copy(right_lane_line)
    right_lane_solve_bottom = np.copy(right_lane_line)
    
    right_lane_solve_top[1] = right_lane_solve_top[1] - global_top_y
    right_lane_solve_bottom[1] = right_lane_solve_bottom[1] - imshape[0]
    
    right_top_x = np.roots(right_lane_solve_top)
    right_bottom_x = np.roots(right_lane_solve_bottom)
    
    left_lane_solve_top = np.copy(left_lane_line)
    left_lane_solve_bottom = np.copy(left_lane_line)
    
    left_lane_solve_top[1] = left_lane_solve_top[1] - global_top_y
    left_lane_solve_bottom[1] = left_lane_solve_bottom[1] - imshape[0]
    
    left_top_x = np.roots(left_lane_solve_top)
    left_bottom_x = np.roots(left_lane_solve_bottom)
    
    cv2.line(img, (right_top_x, global_top_y), (right_bottom_x, imshape[0]), color, thickness)
    cv2.line(img, (left_top_x, global_top_y), (left_bottom_x, imshape[0]), color, thickness)

def get_slope(x1,y1,x2,y2):
    return (y2-y1)/(x2-x1)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

Implementation of single frame processing.

In [3]:
def process_image(image):
    # I. Convert to grayscale
    
    image_gray = grayscale(image)
    
    # II. Build masks for yellow and white pixels
    
    # image in HSV color space
    image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # ranges of "yellow"
    lower_yellow = np.array([20, 100, 100], dtype="uint8")
    upper_yellow = np.array([30, 255, 255], dtype="uint8")

    mask_yellow = cv2.inRange(image_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(image_gray, 200, 255)
    
    # combine masks together
    mask_target = cv2.bitwise_or(mask_white, mask_yellow)
    
    # III. Isolate white and yellow pixels
    
    image_target = cv2.bitwise_and(image_gray, mask_target)
    
    # IV. Apply gaussian blur to supress the noise
    
    kernel_size = 5
    image_target_blurred = gaussian_blur(image_target, kernel_size)
    
    # V. Apply canny edge detection
    
    low_threshold = 50
    high_threshold = 150
    
    image_edged = canny(image_target_blurred, low_threshold, high_threshold)
    
    # VI. Isolate region of intrest
    
    imshape = image.shape
    
    # building verticies of the region
    lower_left = [imshape[1]/9, imshape[0]]
    lower_right = [imshape[1]-imshape[1]/9, imshape[0]]
    top_left = [imshape[1]/2-imshape[1]/8, imshape[0]/2+imshape[0]/10]
    top_right = [imshape[1]/2+imshape[1]/8, imshape[0]/2+imshape[0]/10]
    
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]
    
    image_roi = region_of_interest(image_edged, vertices)
    
    # VII. Apply Hough transform and detect lines
    
    # distance and angular resolution of the grid in Hough space
    rho = 4
    theta = np.pi/180
    
    # minimum number of intersections in a grid for candidate line to go to output
    threshold = 30
    
    # line params
    min_line_len = 10
    max_line_gap = 40

    image_hough = hough_lines(image_roi, rho, theta, threshold, min_line_len, max_line_gap)
    
    # VIII. Overlay the original image with lanes
    
    image_with_lanes_overlay = weighted_img(image_hough, image, α=0.8, β=1., λ=0.)
    
    return image_with_lanes_overlay

## Test on Images

Iterate through "test_images" directory, apply pipeline to the each individual image and save it to the "out_images" directory.

In [4]:
for image_name in os.listdir("test_images/"):
    image = mpimg.imread("test_images/"+image_name)
    image_with_lanes_overlay = process_image(image)
    mpimg.imsave("out_images/processed_"+image_name, image_with_lanes_overlay)

## Test on Videos

Apply "process_image" to the every frame in test videos and save resulting videos with detected lanes.

In [5]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:05<00:00, 43.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 6.28 s, sys: 236 ms, total: 6.51 s
Wall time: 5.59 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [7]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:16<00:00, 41.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 19.2 s, sys: 952 ms, total: 20.2 s
Wall time: 17.2 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [9]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:10<00:00, 24.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 9.86 s, sys: 528 ms, total: 10.4 s
Wall time: 11.7 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

## My conclusion

The provided solution works for well for videos with straight lanes but in the "challenge" video there are problems with lane detection due to curved lanes in the top of the region of intrest. This happens due to my approach to the classification of hough lines by their slope, so in order to improve the pipeline I'll have to find a way to detect change in the slope of the lane.

It's also worth noting that test videos are recorded in the daylight and from straight point of view. If these conditions will not hold, my approach to find a region of intrest will fail to provide the correct region.

There is also a room for improvement to the approximation of lane lines. They should be approximated not just by regular line, but for some higher degree polynomial. I actually tried that, but could not supress the noise completely in the corners of the region of intrest, so I did not include this code to the review.  